In [ ]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 --executor-memory 3g --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StructType, StructField, IntegerType, StringType

if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())


import pyspark.sql.functions as f

In [ ]:
schema = StructType(fields = [
    StructField("id", IntegerType()),
    StructField("lang", StringType()),
        StructField("desc", StringType()),
])

In [ ]:
df = spark.read\
          .format("json")\
          .option("sep", ",")\
          .load("/labs/slaba02/DO_record_per_line.json")

In [ ]:
df.show(2)
#df.select(F.translate('name', 'Aer', '$!').alias('name'))

In [ ]:
import pyspark.sql.functions as f

df2=df.select(f.lower(f.translate("desc",',.-!?:;№#@%&*=+$0123456789"()','')).alias('desc'),"id","lang")

In [ ]:
df3=df2.select(f.translate("desc","'","").alias('desc'),"id","lang")

In [ ]:
df3.select("desc").take(1)

In [ ]:
import pyspark.sql.functions as f
from pyspark.ml.feature import Tokenizer, HashingTF
#tokenizer = Tokenizer(inputCol="comment_text", outputCol="words")

In [ ]:
df3.show(1)

In [ ]:
tokenizer = Tokenizer(inputCol="desc", outputCol="words")

In [ ]:
#desc_prob.select("desc",f.split("desc", " ").alias("word_list")).show(5, False, True)
#log.select("ua", f.split("ua", " ").alias("word_list")).show(5, False, True)

In [ ]:
dataset2 = tokenizer.transform(df3)

In [ ]:
df4=dataset2.select( "id", "words","lang")

In [ ]:
df4.take(1)

In [ ]:
from pyspark.ml.feature import HashingTF
ht = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)
result = ht.transform(df4)
result.show(truncate=False)

In [ ]:
from pyspark.ml.feature import IDF
idf=IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(result)
rescaledData = idfModel.transform(result)
rescaledData.select("id", "features","lang").show(truncate=False)



In [ ]:
rescaledData.printSchema()

In [ ]:
df5=rescaledData.select("id","lang","rawFeatures","features")

In [ ]:
df5.show(5)

In [ ]:
from pyspark.ml.feature import Normalizer

normalizer = Normalizer(inputCol='features', outputCol='norm')
df6 = normalizer.transform(df5)

In [ ]:
df6.where("id in (23126,21617, 16627, 16704)").show(10)

In [ ]:
df6.where(f.col('lang') == 'ru').alias('j').show(2)

In [ ]:
train = df6.where("id in (21617, 16627, 16704, 23126, 13702, 11556)").select(f.col("id").alias("id2"),"lang",f.col("norm").alias("norm2")).cache() #,23126,21617, 16627, 16704)}, seed=42).cache()
test = train.join(df6, on="lang", how="inner").coalesce(10).cache()

In [ ]:
spark.conf.set('spark.sql.crossJoin.enabled', True)
#21617, 16627, 16704, 23126, 13702, 11556

In [ ]:
dot_udf = f.udf(lambda x, y: float(x.dot(y)), DoubleType())
itog=test.select("id","id2", "norm","norm2",dot_udf('norm', 'norm2').alias('dot')).sort(f.desc('dot')).where(f"id<>id2 and id2=11556").limit(10)
otv=itog.select("id2","id","dot")
otv.show()

In [ ]:
for iddd in [13702, 16627, 16704, 21617, 23126]:
    dot_udf = f.udf(lambda x, y: float(x.dot(y)), DoubleType())
    itog=test.select("id","id2", "norm","norm2",dot_udf('norm', 'norm2').alias('dot')).sort(f.desc('dot')).where(f"id<>id2 and id2={iddd}").limit(10)
    otv=otv.union(itog.select("id2","id","dot"))
       # \
        #.select('id','id2' 'dot')\
  

In [ ]:
otv2=otv.select("id2", "id")


In [ ]:
collected_df = otv2.select("id").where("id2=11556").toPandas()
end_11556=collected_df['id'].values.tolist()

In [ ]:
collected_df = otv2.select("id").where("id2=13702").toPandas()
end_13702=collected_df['id'].values.tolist()

In [ ]:
collected_df = otv2.select("id").where("id2=16627").toPandas()
end_16627=collected_df['id'].values.tolist()

In [ ]:
collected_df = otv2.select("id").where("id2=16704").toPandas()
end_16704=collected_df['id'].values.tolist()

In [ ]:
collected_df = otv2.select("id").where("id2=21617").toPandas()
end_21617=collected_df['id'].values.tolist()

In [ ]:
collected_df = otv2.select("id").where("id2=23126").toPandas()
end_23126=collected_df['id'].values.tolist()

In [ ]:
end= dict({'21617' : end_21617 , '23126' : end_23126 , '16627' : end_16627 , '11556' : end_11556, '16704' : end_16704, '13702' : end_13702  }) 

In [ ]:
import json
collected_df = end
with open('lab02.json', 'w') as outfile:
    json.dump(collected_df , outfile , indent=1)
# itog1.write.json("2лаб")

In [ ]:
import json

In [ ]:
z1= sc.parallelize(otv.map(lambda x: x[2]).countByKey().items()).collect()
z1

In [ ]:
with open('lab02.json', 'w') as f:
    json.dump(otv2 , f, indent=2)
    

In [ ]:
otv.write.format('json').save('lab02.json')

In [ ]:
spark.stop()

df5.show(1)